In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
import nltk
from nltk.stem import WordNetLemmatizer
import sys
import bs4
from bs4 import BeautifulSoup
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
porter=PorterStemmer()
import re
import pandas as pd

In [3]:
doc = open("dictionary.xml","r")
soup = BeautifulSoup(doc,'lxml')

dict_gloss = dict()
for i in soup.find_all("lexelt"): 
    x=(i.get('item'))
    for j in i.find_all("sense"):  
        y=(j.get('gloss'))
        dict_gloss.setdefault(x,[]).append(y) 
        
dict_eg=dict()
for i in soup.find_all("lexelt"): 
    x=(i.get('item'))
    for j in i.find_all("sense"):  
        y=(j.get('examples'))
        dict_eg.setdefault(x,[]).append(y)

In [9]:
def get_trainWord(inpList):
    word=re.sub(r'[^a-zA-Z\.]','',(str((inpList)).split("|"))[0])
    word_sense=re.sub(r'[^\d]','',(str((inpList)).split("|"))[1])
    return word,word_sense

In [8]:
ad_s=('i','I','my','me','mine') # creating custom stop words to be removed
stopwords=nltk.corpus.stopwords.words('english')
stopwords.append(ad_s)

def clean_up(inp):
    clean_dt=[]
    for f in str(inp).split(" "):
        w=re.sub(r'[^a-zA-Z\.\%%]','', f)
        w=w.lower()
        if w not in stopwords:
            if w != '':
                w = porter.stem(w)
                clean_dt.append(w)
    return clean_dt

In [10]:
def create_bigrams(lw):
    bi_words = []
    for i in range(0,len(lw)-1):
        bi_words.append(lw[i]+' '+lw[i+1])
    return bi_words

In [11]:
train = open("train.data","r")
test = open("test.data","r")
validate = open("validate.data","r")
nw_train = train
train_List = [line.split('\n') for line in nw_train]

In [7]:
wordnet_lemmatizer = WordNetLemmatizer()

def overlapcontext(word,sense_num, sentence):

    gloss = []
    gloss = clean_up(dict_gloss[word][sense_num])
    gloss.extend(clean_up(dict_eg[word][sense_num]))
    gloss_bi = create_bigrams(gloss)
    
    sent = clean_up(sentence)
    sent_bi = create_bigrams(sent)
   
    return len(set(gloss).intersection(set(sent))), len(set(gloss_bi).intersection(set(sent_bi)))

In [8]:
def lesk(word, sentence, cnt):
    bestsense = None
    bestsense_bi = None
    maxoverlap = 0
    maxoverlap_bi = 0
    
    for sense_num in range(0,len(dict_gloss[word])):
        overlap, overlap_bi = overlapcontext(word,sense_num,sentence)
        
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = word+'.'+str(sense_num+1)
            
        if (overlap_bi!=0) and (overlap_bi > maxoverlap_bi):
            bestsense_bi = word+'.'+str(sense_num+1)
            
    if overlap_bi>0:
        bestsense = bestsense_bi
        
            
    return bestsense

In [9]:
df_predict=pd.DataFrame(columns=['Given','Predicted'])
df_predict_Eg=pd.DataFrame(columns=('Given','Predicted'))
df_predict_Gloss=pd.DataFrame(columns=('Given','Predicted'))

In [10]:
cnt= 0
for each_entry in train_List:
    w,ws=get_trainWord(each_entry) 
    clean_entry=(clean_up(each_entry))
    predicted = lesk(w,clean_entry,cnt)
    given=w+"."+str(ws)
    df_predict.loc[cnt]=given,predicted
    cnt +=1

In [29]:
def accuracy(df):
    res = (df['Given'] == df['Predicted']).values
    return ((res==True).sum()/float(len(res))) * 100

In [12]:
accuracy(df_predict)

49.77783761949644

Corpus Lesk Algorithm

In [1]:
dict_eg_new = dict()

def get_trainWord_new(inpList):
    word=re.sub(r'[^a-zA-Z\.]','',(str((inpList)).split("|"))[0])
    return word

for each_entry in train_List:
    w = get_trainWord_new(each_entry)
    clean_entry = (clean_up(each_entry))
    dict_eg_new[w] = clean_entry

NameError: name 'train_List' is not defined

In [22]:
wordnet_lemmatizer = WordNetLemmatizer()

def overlapcontext1(word,sense_num, sentence):

    gloss = []
    gloss = clean_up(dict_gloss[word][sense_num])
    gloss.extend(clean_up(dict_eg[word][sense_num]))
    gloss.extend(dict_eg_new[word])
    gloss_bi = create_bigrams(gloss)
    
    sent = clean_up(sentence)
    sent_bi = create_bigrams(sent)
   
    return len(set(gloss).intersection(set(sent))), len(set(gloss_bi).intersection(set(sent_bi)))

In [23]:
def corpuslesk(word, sentence, cnt):
    bestsense = None
    bestsense_bi = None
    maxoverlap = 0
    maxoverlap_bi = 0
    
    for sense_num in range(0,len(dict_gloss[word])):
        overlap, overlap_bi = overlapcontext1(word,sense_num,sentence)
        
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = word+'.'+str(sense_num+1)
            
        if (overlap_bi!=0) and (overlap_bi > maxoverlap_bi):
            bestsense_bi = word+'.'+str(sense_num+1)
            
    if overlap_bi>0:
        bestsense = bestsense_bi
        
            
    return bestsense

In [24]:
val_List = [line.split('\n') for line in validate]
test_List = [line.split('\n') for line in test]

In [25]:
df_predict_val=pd.DataFrame(columns=['Given','Predicted'])
df_predict_Eg_val=pd.DataFrame(columns=('Given','Predicted'))
df_predict_Gloss_val=pd.DataFrame(columns=('Given','Predicted'))

In [26]:
df_predict_test=pd.DataFrame(columns=['Given','Predicted'])
df_predict_Eg_test=pd.DataFrame(columns=('Given','Predicted'))
df_predict_Gloss_test=pd.DataFrame(columns=('Given','Predicted'))

In [27]:
cnt= 0
for each_entry in val_List:
    w,ws=get_trainWord(each_entry) 
    clean_entry=(clean_up(each_entry))
    predicted = corpuslesk(w,clean_entry,cnt)
    given=w+"."+str(ws)
    df_predict_val.loc[cnt]=given,predicted
    cnt +=1

In [30]:
accuracy(df_predict_val)

15.326902465166132

In [31]:
cnt= 0
for each_entry in test_List:
    w,ws=get_trainWord(each_entry) 
    clean_entry=(clean_up(each_entry))
    predicted = corpuslesk(w,clean_entry,cnt)
    given=w+"."+str(ws)
    df_predict_test.loc[cnt]=given,predicted
    cnt +=1

In [32]:
accuracy(df_predict_test)

0.0